سنستعمل اليوم أحد مقاييس المسافة، وهو مقياس المسافة الإقليدية؛ وذلك لاسترجاع أو إيجاد المستندات الأكثر تشابها. والفكرة هنا هي أننا قد نمثل محتوى النص أو أسلوبه أو المشاعر فيه بمتجه رقمي. ويقاس التشابه بين هذه المتجهات كوسيلة لقياس التشابه بين النصوص التي تمثلها. وبدلا من استعمال مصنف النصوص للتمييز بين التصنيفات، سنحاول الوقوف على أكثر الأمثلة تشابها.

لنبدأ بتجهيز بياناتنا.

In [1]:
from text_analytics import TextAnalytics
import os
import pandas as pd

ai = TextAnalytics()
ai.data_dir = os.path.join("..", "data")
print("Done!")

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


Done!


وسنبدأ بالنظر في التشابه المعتمد على المؤلف باستعمال الخصائص الأسلوبية. فدعونا إذن نرفع البيانات من مشروع قوتنبيرق. ولأننا لا نحتاج لقاعدة البيانات كاملة، سنرفع جزءا منها وحسب.

In [2]:
file = os.path.join(ai.data_dir, "stylistics.authorship_1850.gz")
df = pd.read_csv(file, nrows = 2000)
print(df)

         Author                Title  \
0      abbott_j  alexander_the_great   
1      abbott_j  alexander_the_great   
2      abbott_j  alexander_the_great   
3      abbott_j  alexander_the_great   
4      abbott_j  alexander_the_great   
...         ...                  ...   
1995  bennett_a   the_old_wives_tale   
1996  bennett_a   the_old_wives_tale   
1997  bennett_a   the_old_wives_tale   
1998  bennett_a   the_old_wives_tale   
1999  bennett_a   the_old_wives_tale   

                                                   Text  
0     note project gutenberg also has an html versio...  
1     it will be recollected to epirus where her fri...  
2     it would be best to endeavor to effect a landi...  
3     transport his army across the straits the army...  
4     that the true greatness of the soul of alexand...  
...                                                 ...  
1995  the stipendiary achieved marvellously the illu...  
1996  winter overcoat he directed the vast affair of...

والآن سنستعد للعثور على العينات الأكثر تشابها بفعل أمرين: (1) تحويل النص إلى متجهات تمثل الأسلوب، و(2) اختيار عينة عشوائية للنظر فيها. ويعني ذلك أننا نختار جزءا من النص عشوائيا ثم نبحث عن العينات الأكثر شبها به.

In [3]:
import random

x, vocab_size = ai.get_features(df, "style")
sample = random.randint(0,len(df))
print(x)
print(sample)

  (0, 0)	338
  (0, 1)	273
  (0, 2)	256
  (0, 3)	125
  (0, 4)	88
  (0, 5)	121
  (0, 6)	1
  (0, 7)	115
  (0, 8)	48
  (0, 9)	102
  (0, 10)	25
  (0, 11)	118
  (0, 13)	62
  (0, 14)	28
  (0, 15)	32
  (0, 16)	33
  (0, 17)	18
  (0, 18)	8
  (0, 19)	16
  (0, 20)	16
  (0, 21)	35
  (0, 22)	34
  (0, 23)	4
  (0, 24)	27
  (0, 25)	13
  :	:
  (1999, 209)	2
  (1999, 210)	1
  (1999, 212)	2
  (1999, 215)	1
  (1999, 218)	2
  (1999, 220)	1
  (1999, 221)	4
  (1999, 222)	2
  (1999, 224)	3
  (1999, 225)	1
  (1999, 232)	3
  (1999, 236)	1
  (1999, 237)	1
  (1999, 239)	1
  (1999, 242)	1
  (1999, 244)	1
  (1999, 250)	3
  (1999, 252)	4
  (1999, 253)	1
  (1999, 278)	1
  (1999, 281)	2
  (1999, 285)	1
  (1999, 295)	2
  (1999, 301)	5
  (1999, 303)	1
224


لقد أصبحنا نعرف الآن ما نبحث عنه. وها هو الكود البرمجي. ومثل كل مرة، إذا أردت التعمق أكثر، ألق نظرة على التفاصيل الفنية للحزمة *text_analytics*. إننا نستعمل الدالة *linguistic_distance* لإيجاد 5 عينات أخرى تشبه عيناتنا.

المتغير *x* يمرر خصائصنا اللغوية.

والمتغير *y* يمرر بياناتنا الوصفية الخاصة بأسماء المؤلفين.

والمتغير *sample* الجزء النصي العشوائي الذي يجب استعماله للبحث عن المتشابهات. ويمكن أن نعيد تشغيل الكود البرمجي أدناه عددا من المرات لتوليد عينات جديدة بالقدر الذي نريد.

أما المتغير *n* فيحدد عدد الأمثلة المتشابهة التي نستهدف العثور عليها.


In [4]:
sample = random.randint(0,len(df))
y_sample, y_closest = ai.linguistic_distance(x = x, y = df.loc[:,"Author"].values, sample = sample, n = 5)
print(y_sample, y_closest)

bennett_a ['bennett_a', 'bennett_a', 'bennett_a', 'bennett_a', 'bennett_a']


لنكرر ذلك مرة أخرى مع خصائص المحتوى. وهذه المرة سنبدأ برفع متجهنا TF-IDF المدرب مسبقا.

In [5]:
file_phrases = os.path.join(".", ai.data_dir, "sociolinguistics.english_all.gz")
ai.phrases = ai.deserialize("phrases", file_phrases + ".phrases.json")
ai.tfidf_vectorizer = ai.deserialize("tfidf_model", file_phrases + ".tfidf.json")

print("Done!")

Done!


والآن نكرر نفس الكود البرمجي باستعمال خصائص *المحتوى*. وسنلقي نظرة على تغريداتنا من مدن مختلفة.

In [6]:
file = os.path.join(ai.data_dir, "sociolinguistics.english_cities.gz")
df = pd.read_csv(file, nrows = 2000)
print(df)

x, vocab_size = ai.get_features(df, "content")

            City                                               Text
0     washington   you really need to go back to bar tending or ...
1         london   jay finley christ in explains why today is co...
2          lagos   forget if this happened truly it s definitely...
3        toronto   yall i love this skin big thanks to for makin...
4        nairobi   the late brilliant prof ali mazrui explains h...
...          ...                                                ...
1995     atlanta   according to cdc s latest levels of u s flu l...
1996       lagos   list of the roads and bridges that buhari is ...
1997     calgary   instead of condemning the assault of by a uni...
1998     phoenix   also vexing how can we use the experience of ...
1999  washington   just landed in the united kingdom heading to ...

[2000 rows x 2 columns]


In [7]:
sample = random.randint(0,len(df))

y_sample, y_closest = ai.linguistic_distance(x = x, y = df.loc[:,"City"].values, sample = sample, n = 5)
print(y_sample, y_closest)

phoenix ['washington', 'washington', 'miami', 'washington', 'washington']


وهذا كل شيء! 

في هذا التدريب العملي رأينا كيف يمكن الوقوف على النصوص المتشابهة أو استرجاعها باستعمال مقياس مسافة بسيط. وقد بحثنا في أسلوب المؤلف باستعمال كتب من القرن التاسع عشر، وفي المحتوى باستعمال التغريدات. والفكرة الأساسية هنا هي أن النصوص قد تتشابه ببعضها بتلك الأساليب الثلاثة.

فهل تعطي مقاييس المسافة نتائج دقيقة كالمصنفات التي استعملناها مسبقا؟ ربما لا. لكن لا بد أن تتذكر أنها أبسط كثيرا من المصنفات فهي غير موجهة؛ وهذا يعني أنه لا توجد أي بيانات تدريب.


In [8]:
file = os.path.join(ai.data_dir, "economic.hotels_as_reviews.gz")
df = pd.read_csv(file, nrows = 5000)
print(df)

x, vocab_size = ai.get_features(df, "sentiment")

                                       Hotel Rating  \
0                 11th Avenue Hotel & Hostel    low   
1                                3 West Club   high   
2                                  414 Hotel   high   
3     70 park avenue hotel - a Kimpton Hotel   high   
4       A Victory Inn & Suites Phoenix North    low   
...                                      ...    ...   
4995                         The Lenox Hotel   high   
4996                    The Listel Vancouver   high   
4997                         The Loden Hotel   high   
4998                            The Lombardy   high   
4999                      The Lonsdale Hotel    low   

                                                   Text  
0     This hostel is in a very good location, close ...  
1     We had 5 nights here and were unsure as to wha...  
2     This is a small boutique hotel with a nice int...  
3     I stayed at 70 Park Ave Hotel the night before...  
4     I made a reservation. Cancelled 2 hours lat

In [9]:
import random
sample = random.randint(0,len(df))

y_sample, y_closest = ai.linguistic_distance(x = x, y = df.loc[:,"Rating"].values, sample = sample, n = 5)
print(y_sample, y_closest)

high ['high', 'high', 'high', 'high', 'high']
